In [2]:
import pandas as pd

from sklearn.neighbors import NearestNeighbors
import numpy as np
import implicit
from scipy.sparse import csr_matrix, save_npz, load_npz
from tqdm.notebook import trange

In [6]:
data = load_npz("csr_user_item.npz")

In [7]:
data

<76446x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 1920530 stored elements in Compressed Sparse Row format>

In [8]:
%%time


a=0
for i in range(1):
    R = data

    # Создаем и обучаем модель
    model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=10)
    model.fit(R)

    
    # Рекомендуем элементы для пользователя
    user_id = 1488
    recommended, _ = model.recommend(user_id, R[user_id])
    print(recommended)

C:\Users\Александр\AppData\Roaming\Python\Python311\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

[ 840 4052 1882 2416 8141 1392 2771 3332  532  456]
CPU times: total: 3min 14s
Wall time: 19.3 s


In [9]:
uf = model.user_factors

In [10]:
uf.shape

(76446, 20)

In [11]:
nbrs = NearestNeighbors(algorithm='brute', metric = 'cosine').fit(uf)


In [12]:
def predict(i, n_neighb):
    d, ind = nbrs.kneighbors(uf[i].reshape(1,-1), n_neighbors=n_neighb+1)
    print(d)
    print(uf[i])
    print('='*30)
    for i in ind[0][1:]:
        print(i)

In [13]:
predict(1486, 5)

[[0.         0.1634075  0.16666478 0.16891724 0.16894197 0.18025142]]
[  3.3207808   -1.8830144   -1.6475973   10.702188    -0.31295276
  37.14653      0.10665691  27.259357   -12.426449     7.7998204
  -5.5771866   16.650562     4.6062255    7.3677645   21.1869
  15.577896   -22.95684     -6.201458    -2.4522011    5.6838007 ]
6122
17012
36907
20308
11782


In [14]:
a = np.nonzero(data[23736])[1]

b = np.nonzero(data[1486])[1]
k=0
for i in np.concatenate([a,b]):
    if i in a and i in b:
        k+=1
print(k//2)

81


In [18]:
datalil = data.tolil()

In [19]:
datalil

<76446x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 1921287 stored elements in List of Lists format>

In [20]:
def predict_cnt(i, n_neighb, net):
    d, ind = nbrs.kneighbors(uf[i].reshape(1,-1), n_neighbors=n_neighb+1)
    a = net[ind[0][0]]
    for ii in ind[0][1:]:
        a+=net[ii]
        #print('-'*30)
    for j in range(a.shape[1]):
        if a[0, j] != 0:  # Добавляем только ненулевые значения
            net[i, j] = a[0, j]
    


In [21]:
%%time
for i in trange(100):
    predict_cnt(i, 5, datalil)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: total: 11.5 s
Wall time: 11.4 s


<76446x31916 sparse matrix of type '<class 'numpy.float64'>'
	with 1924213 stored elements in List of Lists format>

In [62]:
VIDEO = pd.read_csv("data/24tv_video_2024_11_11.csv")

In [ ]:
uf

In [70]:
VIDEO

,id,uuid,external_stat_id,user,profile,device,device_type,video,episode,history,...,started_at,ended_at,preview,session_id,library_id,access_token,app_version,cause,content,top_parent
0,UInt64,Nullable(UUID),Nullable(UUID),UInt64,Nullable(FixedString(36)),Nullable(FixedString(36)),Nullable(String),UInt32,UInt32,Nullable(UInt64),...,DateTime,Nullable(DateTime),Nullable(UInt8),Nullable(UUID),Nullable(UInt32),Nullable(String),Nullable(UInt32),Nullable(String),Nullable(UInt64),Nullable(UInt64)
1,1056990830854268768,97943a99-5407-4dca-930f-d2057021d969,\N,16821584,45c35ba0-69bb-4dbd-99af-c25037e9337e,ae857ffb-a744-4323-9dc3-2f23b733c0fe,tv,0,0,1056990830774560372,...,2024-11-11 00:00:02,2024-11-11 00:06:14,0,97943a99-5407-4dca-930f-d2057021d969,8,a2c32020368bf91b9b1c7734b802f1d52febef88,320462,\N,546590812041904598,546321152977404315
2,1056990837064925237,8c4e0084-da2c-46da-b141-6f20d0ce31ad,\N,10586519,d3e89d3b-31fc-44b4-aa35-4c45e0b16980,c2af1171-443e-4b57-b80a-67cc02e5a67e,stb,0,0,\N,...,2024-11-11 00:00:03,2024-11-11 00:00:03,1,8c4e0084-da2c-46da-b141-6f20d0ce31ad,8,4dab0abe45cb2de45c861657fcee86e9dda93839,3552,\N,852503324376609050,852503316713302252
3,1056990837351001036,69423891-8d07-4dab-aaeb-900078f177b9,\N,17636978,09ab3d2f-2cbc-4812-9730-203004731f1a,2639eb3a-c0c5-4302-a449-4ffcce770a87,stb,0,0,1056990965091705690,...,2024-11-11 00:00:03,2024-11-11 00:02:04,0,69423891-8d07-4dab-aaeb-900078f177b9,8,f69c1801698324a3976eb77d9426bc5c1fa08b2c,3626,\N,547476653031293513,546312519552074714
4,1056990842169545753,1552a414-e2af-4c71-aaa2-5a0fea1a4e89,\N,10586519,d3e89d3b-31fc-44b4-aa35-4c45e0b16980,c2af1171-443e-4b57-b80a-67cc02e5a67e,stb,0,0,\N,...,2024-11-11 00:00:04,2024-11-11 00:00:04,1,1552a414-e2af-4c71-aaa2-5a0fea1a4e89,8,4dab0abe45cb2de45c861657fcee86e9dda93839,3552,\N,852503324376609050,852503316713302252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20133,1057344547674041516,d3e80c26-b64d-440c-a697-a020fff4175a,\N,18013555,0d9b3703-4693-4cac-96a5-ae50cec19f1d,b1b37492-7a2c-4fd4-9421-8d5b19a35c89,stb,0,0,1057005576805563500,...,2024-11-11 23:59:18,2024-11-12 00:22:30,0,d3e80c26-b64d-440c-a697-a020fff4175a,8,22bda082753e55e1974328f679f966924bc2a678,3619,\N,546592870862160781,546592793108152401
20134,1057344604727032729,42071226-cc34-4aed-a68d-baec06b1e288,\N,17946789,016d3f0a-70b0-4974-b00b-36183c2c3da3,e2bcbb16-c688-447f-be99-19ce07e75752,tv,0,0,1057344501131391191,...,2024-11-11 23:59:32,2024-11-12 00:45:59,0,42071226-cc34-4aed-a68d-baec06b1e288,8,5f6739e3b13f82455c10bdb19cae0255af074c84,324514,\N,900305172996443183,893939246198514494
20135,1057344650293952199,69cf448e-cad2-4274-a8fe-0470fa31b90b,\N,10830926,402b0391-0748-45c6-83d2-b745e33d5406,12c2490e-e202-47cb-bc96-daad335badc2,tv,0,0,1057344650215350569,...,2024-11-11 23:59:43,2024-11-12 00:48:24,0,69cf448e-cad2-4274-a8fe-0470fa31b90b,1,487ff2ce5b8a247df3e4987cba67fcd129ce6c19,324514,\N,904170163763909279,904170150282979823
20136,1057344652125909572,327a8455-23ad-47eb-b3eb-3c31b800feb9,\N,18148002,dbdbf499-fd3e-49e2-a8c2-17465f59ba60,78acc625-dea0-467d-b66c-a41ba1e5e4f2,phone,0,0,1057344652025678704,...,2024-11-11 23:59:44,2024-11-12 00:19:05,0,327a8455-23ad-47eb-b3eb-3c31b800feb9,8,6e47fab5c22afa454634fbd518a40e4e188a5223,\N,\N,546317004336791957,546316996308894013


In [ ]:
import numpy as np
from scipy.sparse import lil_matrix, coo_matrix

# Пример данных
data = {
    'row': [0, 0, 1, 1, 2, 2],
    'col': [0, 1, 0, 1, 0, 1],
    'value': [1, 2, 3, 4, 5, 6]
}

# Извлечение данных
rows = np.array(data['row'])
cols = np.array(data['col'])
values = np.array(data['value'])

# Создание COO матрицы
coo = coo_matrix((values, (rows, cols)), shape=(3, 2))

# Преобразование в LIL матрицу
lil_mat = coo.tolil()

# Вывод результата
print("LIL Matrix:")
print(lil_mat)